In [9]:
import base64
import pandas as pd

def create_onedrive_directdownload (onedrive_link):
    data_bytes64 = base64.b64encode(bytes(onedrive_link, 'utf-8'))
    data_bytes64_String = data_bytes64.decode('utf-8').replace('/','_').replace('+','-').rstrip("=")
    resultUrl = f"https://api.onedrive.com/v1.0/shares/u!{data_bytes64_String}/root/content"
    return resultUrl

onedrive_url = "https://1drv.ms/x/s!Avnf_07fKpTXiVCCyUr_P40FhuNf?e=wL2j3t"
converted_url = create_onedrive_directdownload(onedrive_url)
pd.read_excel(converted_url)


,testdata,werkt dit
0,20,11


In [3]:
print("fuck off")

fuck off
